In [1]:
from src.templates.email_template import EmailTemplate
from src.templates.product_template import ProductTemplate
from src.utils.config_loader import load_config
from src.assistant import Assistant

In [2]:
config = load_config("/Users/raviguntakala/Desktop/Personal_Projects/synthetic_data_geneation/config/system_config.yaml")

In [3]:
email_template = EmailTemplate(config)
product_template = ProductTemplate(config)
llm_client = Assistant()

In [4]:
# prompt,output_model = email_template.build_prompt(num_samples=10)
prompt,output_model = product_template.build_prompt(num_samples=10)


In [5]:
print(prompt)

Generate EXACTLY 10 distinct and realistic product descriptions for an e-commerce setting.

        Ensure variety across the generated product descriptions using different combinations of:
        - Product Category: Use a variety from ['electronics', 'furniture', 'clothing', 'kitchenware', 'beauty', 'sports', 'toys', 'books', 'pet supplies', 'jewelry', 'office supplies', 'automotive', 'food and beverages']. Assign a relevant category to each product.
        - Price Tier: Use different tiers from ['budget', 'mid_range', 'premium', 'luxury'], generating a specific price within the corresponding range: {'budget': [5, 50], 'mid_range': [51, 200], 'premium': [201, 1000], 'luxury': [1001, 10000]}.
        - Complexity Levels: Generate descriptions representing DIFFERENT complexity levels, adhering strictly to the following guidelines for description length, feature count, and specification count:
        - simple: Basic description with fundamental features (approx. 30-70 words, 2-4 featu

In [5]:
print(output_model)

<class 'src.templates.product_template.ProductOutputList'>


In [6]:
response = llm_client.generate_response(prompt=prompt,output_pydantic=output_model)

In [8]:
print(response.content)

{
  "emails": [
    {
      "closing": "Thanks,",
      "customer_name": "Jane Doe",
      "issue_type": "delivery_delay",
      "complexity_level": "simple",
      "subject": "Order Delivery Delay",
      "greeting": "Hello,",
      "body": "My order (Order #12345) was supposed to arrive yesterday. I haven't received any updates. Can you please check the status and provide an estimated delivery date? It included a new set of cookware."
    },
    {
      "closing": "Best regards,",
      "customer_name": "John Smith",
      "issue_type": "order_status",
      "complexity_level": "medium",
      "subject": "Inquiry about Order #67890",
      "greeting": "Dear Support Team,",
      "body": "I placed an order for a leather jacket (Order #67890) on July 10th. The tracking information hasn't been updated in three days. Could you please provide more detailed information about its current location and when I can expect it to arrive? I'm hoping it arrives before my trip next week."
    },
   

In [7]:
print(response.content)

{
  "products": [
    {
      "complexity_level": "simple",
      "category": "kitchenware",
      "description": "This stainless steel whisk is perfect for all your baking needs. Its durable construction ensures long-lasting use, and the ergonomic handle provides a comfortable grip. Easily whip cream, beat eggs, or mix batter with this essential kitchen tool.",
      "product_name": "Stainless Steel Whisk",
      "features": [
        "Durable stainless steel",
        "Ergonomic handle"
      ],
      "tagline": "Whip up delicious treats with ease!",
      "price": 8.99
    },
    {
      "complexity_level": "medium",
      "category": "electronics",
      "description": "The Echo Dot (5th Gen) delivers improved audio and a sleek design. Control your smart home, stream music, and get information hands-free. With enhanced bass and clear vocals, enjoy a richer audio experience. Plus, it's designed with privacy in mind.",
      "product_name": "Echo Dot (5th Gen)",
      "features": [
 

In [13]:
print(prompt)

Generate a realistic customer email about a missing_item for a food order.
        
        The email should be at simple complexity level:
        - simple: Brief, direct inquiry with basic language
        - medium: Some details and context with average vocabulary
        - complex: Multiple questions or concerns with more sophisticated language
        - very_complex: Detailed situation description with advanced vocabulary, multiple references to order details, and complex requests
        
        Format your response according to this Pydantic model:
        
        ```python
        class EmailOutput(BaseModel):
            subject: str  # Email subject line
            greeting: str  # Email greeting (e.g., "Hello," or "Dear Support Team,")
            body: str  # Main body explaining the situation and inquiry
            closing: str  # Email closing (e.g., "Thank you," or "Best regards,")
            customer_name: str  # Name of the customer
            complexity_level: st

In [27]:
import json
json.loads(response.content)

{'emails': [{'issue_type': 'delivery_delay',
   'closing': 'Thanks,',
   'greeting': 'Hi,',
   'subject': 'Order #12345 Delivery Delay',
   'body': "My order, #12345, was supposed to arrive today, but the tracking hasn't updated in two days. Can you check on its status?",
   'customer_name': 'Jane Doe',
   'complexity_level': 'simple'},
  {'issue_type': 'order_status',
   'closing': 'Best,',
   'greeting': 'Hello,',
   'subject': 'Inquiry about Order #67890',
   'body': "I placed an order (#67890) for a new TV stand a week ago and haven't received any shipping updates. Could you please provide an update on its current status?",
   'customer_name': 'John Smith',
   'complexity_level': 'medium'},
  {'issue_type': 'wrong_item',
   'closing': 'Sincerely,',
   'greeting': 'Dear Support Team,',
   'subject': 'Incorrect Item Received - Order #24680',
   'body': 'I received the wrong item in my order #24680. I ordered a blue sweater, but I received a red t-shirt instead. Please advise on how t

In [ ]:
import requests

response = requests.post(
    "http://localhost:8000/generate",
    json={
        "template_name": "products",
        "count": 1,
        "output_format": "json"
    }
)
print(response.json())

{'status': 'success', 'output_path': 'data/output/emails/emails.json', 'count': 1, 'template_name': 'emails', 'output_format': 'json'}


In [ ]:
import requests

response = requests.post(
    "http://localhost:8000/generate",
    json={
        "template_name": "emails",
        "count": 1,
        "output_format": "json"
    }
)
print(response.json())

{'status': 'success', 'output_path': 'data/output/products/products.json', 'count': 1, 'template_name': 'products', 'output_format': 'json'}


In [4]:
import requests
response = requests.post(
    "http://localhost:8000/generate-batch",
    json={
        "template_name": "emails",
        "count": 3,
        "output_format": "json",
        "batch" : True
    }
)
print(response.json())

{'status': 'success', 'output_path': ['data/output/emails/emails.json', 'data/output/emails/emails.json'], 'requested_count': 3, 'generated_count': 30, 'template_name': 'emails', 'output_format': 'json', 'batch_mode': True}


In [96]:
config['email_templates']['response_fields']

{'subject': 'str',
 'greeting': 'str',
 'body': 'str',
 'closing': 'str',
 'customer_name': 'str',
 'complexity_level': 'str',
 'issue_type': 'str',
 'order_id': 'str',
 'product_name': 'str',
 'product_type': 'str'}

In [97]:
config['product_templates']['response_fields']

{'product_name': 'str',
 'tagline': 'str',
 'description': 'str',
 'features': 'List[str]',
 'price': 'float',
 'complexity_level': 'str',
 'category': 'str'}

In [1]:
from src.utils.pydantic_generator import create_pydantic_models
 
from src.utils.config_loader import load_config
file = "/Users/raviguntakala/Desktop/Personal_Projects/synthetic_data_geneation/config/system_config.yaml"
config = load_config(file)

Logfire project URL: ]8;id=59294;https://logfire-us.pydantic.dev/ravi/testing-fastapi\https://logfire-us.pydantic.dev/ravi/testing-fastapi]8;;\


In [ ]:
EmailOutput, EmailOutputList = create_pydantic_models(
    "EmailOutput",
    config['email_templates']['response_fields'],
    docstring="Pydantic model for email output",
    list_field_name= "emails"
)

13:00:13.300 Successfully created item model: EmailOutput
13:00:13.302 Successfully created list model: EmailOutputList


In [2]:
ProductOutput, ProductOutputList = create_pydantic_models(
    "ProductOutput",
    config['product_templates']['response_fields'],
    docstring="Pydantic model for product output",
    list_field_name= "products"
)


13:09:11.269 Successfully created item model: ProductOutput
13:09:11.272 Successfully created list model: ProductOutputList


In [3]:
ProductOutput.model_json_schema()

{'description': 'Pydantic model for product output',
 'properties': {'product_name': {'title': 'Product Name', 'type': 'string'},
  'tagline': {'title': 'Tagline', 'type': 'string'},
  'description': {'title': 'Description', 'type': 'string'},
  'features': {'items': {'type': 'string'},
   'title': 'Features',
   'type': 'array'},
  'price': {'title': 'Price', 'type': 'number'},
  'complexity_level': {'title': 'Complexity Level', 'type': 'string'},
  'category': {'title': 'Category', 'type': 'string'}},
 'required': ['product_name',
  'tagline',
  'description',
  'features',
  'price',
  'complexity_level',
  'category'],
 'title': 'ProductOutput',
 'type': 'object'}

In [90]:
EmailOutputList.model_json_schema()

{'$defs': {'EmailOutput': {'description': 'Pydantic model for email output',
   'properties': {'subject': {'title': 'Subject', 'type': 'string'},
    'greeting': {'title': 'Greeting', 'type': 'string'},
    'body': {'title': 'Body', 'type': 'string'},
    'closing': {'title': 'Closing', 'type': 'string'},
    'customer_email': {'title': 'Customer Email', 'type': 'string'},
    'complexity_level': {'title': 'Complexity Level', 'type': 'string'},
    'issue_type': {'title': 'Issue Type', 'type': 'string'},
    'order_id': {'title': 'Order Id', 'type': 'string'},
    'product_name': {'title': 'Product Name', 'type': 'string'},
    'product_type': {'title': 'Product Type', 'type': 'string'}},
   'required': ['subject',
    'greeting',
    'body',
    'closing',
    'customer_email',
    'complexity_level',
    'issue_type',
    'order_id',
    'product_name',
    'product_type'],
   'title': 'EmailOutput',
   'type': 'object'}},
 'description': 'Pydantic model for a list of EmailOutput ins

In [83]:
# --- Existing EmailOutput definition ---
class EmailOutput(BaseModel):
    """Pydantic model for email output"""
    subject: str
    greeting: str
    body: str
    closing: str
    customer_name: str
    complexity_level: str
    issue_type: str

# --- NEW: Pydantic model for a list of email outputs ---
class EmailOutputList(BaseModel):
    """Pydantic model for a list of email outputs"""
    emails: List[EmailOutput]
EmailOutputList.model_json_schema()

{'$defs': {'EmailOutput': {'description': 'Pydantic model for email output',
   'properties': {'subject': {'title': 'Subject', 'type': 'string'},
    'greeting': {'title': 'Greeting', 'type': 'string'},
    'body': {'title': 'Body', 'type': 'string'},
    'closing': {'title': 'Closing', 'type': 'string'},
    'customer_name': {'title': 'Customer Name', 'type': 'string'},
    'complexity_level': {'title': 'Complexity Level', 'type': 'string'},
    'issue_type': {'title': 'Issue Type', 'type': 'string'}},
   'required': ['subject',
    'greeting',
    'body',
    'closing',
    'customer_name',
    'complexity_level',
    'issue_type'],
   'title': 'EmailOutput',
   'type': 'object'}},
 'description': 'Pydantic model for a list of email outputs',
 'properties': {'emails': {'items': {'$ref': '#/$defs/EmailOutput'},
   'title': 'Emails',
   'type': 'array'}},
 'required': ['emails'],
 'title': 'EmailOutputList',
 'type': 'object'}